In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout,LSTM

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Raw_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/only data/raw_data.npz'

Theta_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time frequency domain/RAW/Delta_merged_features.npz'



# RAW = np.load(Raw_data, allow_pickle=True)  # Allow loading pickled object arrays
# X = RAW['X'].astype('float64')
# Y = RAW['Y'].astype('float64')
# group = RAW['Group'].astype('float64')

This is a good performing model

In [ ]:
# %%capture
# !pip install wandb

# CNN-LSTM

In [7]:
%%capture
!pip install tensorflow_addons

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Delta/CNN_LSTM/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Delta/CNN_LSTM/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


In [17]:
global_model, metrics_df = federated_learning(Theta_data)


Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 37, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 256)           0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 9s 59ms/step - loss: 1.7015 - accuracy: 0.4954 - val_loss: 1.6963 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 0s 15ms/step - loss: 1.6913 - accuracy: 0.5038 - val_loss: 1.6861 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 0s 15ms/step - loss: 1.6811 - accuracy: 0.5038 - val_loss: 1.6760 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 0s 15ms/step - loss: 1.6710 - accuracy: 0.5038 - val_loss: 1.6660 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 0s 15ms/step - loss: 1.6610 - accuracy: 0.5038 - val_loss: 1.6561 - val_accuracy: 0.4849
Epoch 6/100
29/29 [==============================] - 0s 15ms/step - loss: 1.6511 - accuracy: 0.5038 - val_loss: 1.6463 - val_accuracy: 0.4849
Epoch 7/100
29/29 [==============================] - 0s 15ms/step - loss: 1.6412 - accuracy: 0.5038 - val_loss: 1.6366 - val_accuracy: 0.4849
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 74ms/step - loss: 1.7018 - accuracy: 0.5011 - val_loss: 1.6966 - val_accuracy: 0.4955
Epoch 2/100
28/28 [==============================] - 1s 21ms/step - loss: 1.6918 - accuracy: 0.5011 - val_loss: 1.6868 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 1s 20ms/step - loss: 1.6819 - accuracy: 0.5011 - val_loss: 1.6770 - val_accuracy: 0.4955
Epoch 4/100
28/28 [==============================] - 1s 19ms/step - loss: 1.6722 - accuracy: 0.5011 - val_loss: 1.6674 - val_accuracy: 0.4955
Epoch 5/100
28/28 [==============================] - 1s 25ms/step - loss: 1.6624 - accuracy: 0.5011 - val_loss: 1.6578 - val_accuracy: 0.4955
Epoch 6/100
28/28 [==============================] - 1s 20ms/step - loss: 1.6528 - accuracy: 0.5011 - val_loss: 1.6483 - val_accuracy: 0.4955
Epoch 7/100
28/28 [==============================] - 0s 17ms/step - loss: 1.6432 - accuracy: 0.5011 - val_loss: 1.6389 - val_accuracy: 0.4955
Epoch 8/100
28/28 

...

29/29 [==============================] - 8s 62ms/step - loss: 0.5667 - accuracy: 0.7449 - val_loss: 0.7750 - val_accuracy: 0.5280
Epoch 2/100
29/29 [==============================] - 0s 15ms/step - loss: 0.5622 - accuracy: 0.7406 - val_loss: 0.7918 - val_accuracy: 0.5226
Epoch 3/100
29/29 [==============================] - 1s 23ms/step - loss: 0.5728 - accuracy: 0.7398 - val_loss: 0.7676 - val_accuracy: 0.5690
Epoch 4/100
29/29 [==============================] - 1s 25ms/step - loss: 0.5475 - accuracy: 0.7505 - val_loss: 0.7656 - val_accuracy: 0.5959
Epoch 5/100
29/29 [==============================] - 0s 16ms/step - loss: 0.5527 - accuracy: 0.7355 - val_loss: 0.7723 - val_accuracy: 0.5022
Epoch 6/100
29/29 [==============================] - 1s 23ms/step - loss: 0.5479 - accuracy: 0.7524 - val_loss: 0.7608 - val_accuracy: 0.6369
Epoch 7/100
29/29 [==============================] - 0s 16ms/step - loss: 0.5487 - accuracy: 0.7408 - val_loss: 0.7601 - val_accuracy: 0.6131
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 7s 66ms/step - loss: 0.5662 - accuracy: 0.7385 - val_loss: 0.7583 - val_accuracy: 0.6199
Epoch 2/100
28/28 [==============================] - 1s 21ms/step - loss: 0.5689 - accuracy: 0.7462 - val_loss: 0.7542 - val_accuracy: 0.5860
Epoch 3/100
28/28 [==============================] - 1s 21ms/step - loss: 0.5656 - accuracy: 0.7422 - val_loss: 0.7518 - val_accuracy: 0.5430
Epoch 4/100
28/28 [==============================] - 1s 21ms/step - loss: 0.5551 - accuracy: 0.7507 - val_loss: 0.7569 - val_accuracy: 0.5475
Epoch 5/100
28/28 [==============================] - 1s 20ms/step - loss: 0.5464 - accuracy: 0.7575 - val_loss: 0.7638 - val_accuracy: 0.5385
Epoch 6/100
28/28 [==============================] - 1s 24ms/step - loss: 0.5503 - accuracy: 0.7462 - val_loss: 0.7482 - val_accuracy: 0.5633
Epoch 7/100
28/28 [==============================] - 1s 37ms/step - loss: 0.5462 - accuracy: 0.7581 - val_loss: 0.7379 - val_accuracy: 0.6312
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 10s 57ms/step - loss: 0.5914 - accuracy: 0.7305 - val_loss: 0.7653 - val_accuracy: 0.5589
Epoch 2/100
31/31 [==============================] - 1s 24ms/step - loss: 0.5752 - accuracy: 0.7302 - val_loss: 0.7622 - val_accuracy: 0.5610
Epoch 3/100
31/31 [==============================] - 1s 17ms/step - loss: 0.5713 - accuracy: 0.7444 - val_loss: 0.7632 - val_accuracy: 0.5320
Epoch 4/100
31/31 [==============================] - 1s 26ms/step - loss: 0.5838 - accuracy: 0.7230 - val_loss: 0.7604 - val_accuracy: 0.5764
Epoch 5/100
31/31 [==============================] - 1s 16ms/step - loss: 0.5750 - accuracy: 0.7271 - val_loss: 0.7646 - val_accuracy: 0.5444
Epoch 6/100
31/31 [==============================] - 1s 24ms/step - loss: 0.5656 - accuracy: 0.7401 - val_loss: 0.7550 - val_accuracy: 0.5878
Epoch 7/100
31/31 [==============================] - 1s 18ms/step - loss: 0.5662 - accuracy: 0.7411 - val_loss: 0.7525 - val_accuracy: 0.5806
Epoch 8/100
31/31

In [18]:

metrics_df.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.548,0.623,0.241,0.348,0.241,0.854,0.095
1,1,0.557,0.617,0.301,0.405,0.301,0.814,0.114
2,2,0.555,0.572,0.438,0.496,0.438,0.673,0.110
3,0,0.564,0.708,0.219,0.335,0.219,0.910,0.129
4,1,0.572,0.618,0.377,0.468,0.377,0.767,0.144
5,2,0.584,0.613,0.458,0.524,0.458,0.711,0.169
6,0,0.590,0.715,0.298,0.421,0.298,0.881,0.179
7,1,0.585,0.652,0.364,0.467,0.364,0.805,0.169
8,2,0.614,0.598,0.701,0.645,0.701,0.528,0.229
9,0,0.595,0.596,0.593,0.594,0.593,0.598,0.191


In [19]:
metrics_df.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/TF Domain/CNN_LSTM/Delta_tf_CNN_LSTM.csv', index = False)

# CNN

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(LSTM(256, return_sequences=True))
    # model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Delta/CNN/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Delta/CNN/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [9]:
global_model_CNN, metrics_df_CNN = federated_learning(Theta_data)

Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 37, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 256)           0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 18s 60ms/step - loss: 1.9607 - accuracy: 0.5046 - val_loss: 1.9516 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 1s 39ms/step - loss: 1.9414 - accuracy: 0.5038 - val_loss: 1.9332 - val_accuracy: 0.5409
Epoch 3/100
29/29 [==============================] - 1s 46ms/step - loss: 1.9228 - accuracy: 0.5054 - val_loss: 1.9151 - val_accuracy: 0.5560
Epoch 4/100
29/29 [==============================] - 1s 28ms/step - loss: 1.9045 - accuracy: 0.5084 - val_loss: 1.8972 - val_accuracy: 0.5485
Epoch 5/100
29/29 [==============================] - 1s 29ms/step - loss: 1.8865 - accuracy: 0.5164 - val_loss: 1.8796 - val_accuracy: 0.5474
Epoch 6/100
29/29 [==============================] - 1s 37ms/step - loss: 1.8688 - accuracy: 0.5242 - val_loss: 1.8623 - val_accuracy: 0.5453
Epoch 7/100
29/29 [==============================] - 1s 33ms/step - loss: 1.8513 - accuracy: 0.5229 - val_loss: 1.8453 - val_accuracy: 0.5453
Epoch 8/100
29/29

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 1s 20ms/step - loss: 1.9421 - accuracy: 0.5023 - val_loss: 1.9343 - val_accuracy: 0.5362
Epoch 3/100
28/28 [==============================] - 0s 15ms/step - loss: 1.9239 - accuracy: 0.5085 - val_loss: 1.9167 - val_accuracy: 0.5260
Epoch 4/100
28/28 [==============================] - 0s 14ms/step - loss: 1.9060 - accuracy: 0.5170 - val_loss: 1.8994 - val_accuracy: 0.5283
Epoch 5/100
28/28 [==============================] - 0s 15ms/step - loss: 1.8884 - accuracy: 0.5170 - val_loss: 1.8823 - val_accuracy: 0.5226
Epoch 6/100
28/28 [==============================] - 0s 14ms/step - loss: 1.8711 - accuracy: 0.5198 - val_loss: 1.8655 - val_accuracy: 0.5238
Epoch 7/100
28/28 [==============================] - 0s 14ms/step - loss: 1.8542 - accuracy: 0.5255 - val_loss: 1.8490 - val_accuracy: 0.5226
Epoch 8/100
28/28 [==============================] - 0s 13ms/step - loss: 1.8376 - accuracy: 0.5396 - val_loss: 1.8328 - val_accuracy: 0.5181
Epoch 9/100
28/28 

...

29/29 [==============================] - 1s 19ms/step - loss: 0.5048 - accuracy: 0.8327 - val_loss: 0.8624 - val_accuracy: 0.4914
Epoch 3/100
29/29 [==============================] - 1s 26ms/step - loss: 0.5063 - accuracy: 0.8246 - val_loss: 0.8485 - val_accuracy: 0.5054
Epoch 4/100
29/29 [==============================] - 1s 25ms/step - loss: 0.5098 - accuracy: 0.8367 - val_loss: 0.8453 - val_accuracy: 0.6056
Epoch 5/100
29/29 [==============================] - 0s 15ms/step - loss: 0.4940 - accuracy: 0.8481 - val_loss: 0.8423 - val_accuracy: 0.5291
Epoch 6/100
29/29 [==============================] - 0s 16ms/step - loss: 0.4986 - accuracy: 0.8303 - val_loss: 0.8619 - val_accuracy: 0.5022
Epoch 7/100
29/29 [==============================] - 0s 15ms/step - loss: 0.4903 - accuracy: 0.8446 - val_loss: 0.8562 - val_accuracy: 0.5075
Epoch 8/100
29/29 [==============================] - 0s 16ms/step - loss: 0.4852 - accuracy: 0.8462 - val_loss: 0.8225 - val_accuracy: 0.5927
Epoch 9/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 0s 13ms/step - loss: 0.5206 - accuracy: 0.8345 - val_loss: 0.8691 - val_accuracy: 0.5079
Epoch 3/100
28/28 [==============================] - 1s 22ms/step - loss: 0.5081 - accuracy: 0.8325 - val_loss: 0.8522 - val_accuracy: 0.5181
Epoch 4/100
28/28 [==============================] - 1s 21ms/step - loss: 0.5047 - accuracy: 0.8362 - val_loss: 0.8214 - val_accuracy: 0.5588
Epoch 5/100
28/28 [==============================] - 0s 14ms/step - loss: 0.5066 - accuracy: 0.8415 - val_loss: 0.8305 - val_accuracy: 0.5328
Epoch 6/100
28/28 [==============================] - 0s 15ms/step - loss: 0.4927 - accuracy: 0.8540 - val_loss: 0.8473 - val_accuracy: 0.5204
Epoch 7/100
28/28 [==============================] - 0s 13ms/step - loss: 0.4868 - accuracy: 0.8520 - val_loss: 0.8310 - val_accuracy: 0.5373
Epoch 8/100
28/28 [==============================] - 0s 15ms/step - loss: 0.4863 - accuracy: 0.8492 - val_loss: 0.8437 - val_accuracy: 0.5328
Epoch 9/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 1s 21ms/step - loss: 0.5268 - accuracy: 0.8183 - val_loss: 0.8403 - val_accuracy: 0.5021
Epoch 3/100
31/31 [==============================] - 0s 16ms/step - loss: 0.5227 - accuracy: 0.8276 - val_loss: 0.8631 - val_accuracy: 0.4979
Epoch 4/100
31/31 [==============================] - 1s 21ms/step - loss: 0.5355 - accuracy: 0.8142 - val_loss: 0.8539 - val_accuracy: 0.5062
Epoch 5/100
31/31 [==============================] - 1s 29ms/step - loss: 0.5318 - accuracy: 0.8165 - val_loss: 0.8392 - val_accuracy: 0.5227
Epoch 6/100
31/31 [==============================] - 1s 19ms/step - loss: 0.5193 - accuracy: 0.8251 - val_loss: 0.8514 - val_accuracy: 0.5186
Epoch 7/100
31/31 [==============================] - 1s 18ms/step - loss: 0.5205 - accuracy: 0.8274 - val_loss: 0.8454 - val_accuracy: 0.5207
Epoch 8/100
31/31 [==============================] - 1s 26ms/step - loss: 0.5189 - accuracy: 0.8344 - val_loss: 0.8138 - val_accuracy: 0.5610
Epoch 9/100
31/31 

In [10]:
metrics_df_CNN.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.539,0.556,0.380,0.452,0.380,0.697,0.077
1,1,0.554,0.586,0.367,0.451,0.367,0.740,0.107
2,2,0.552,0.576,0.396,0.469,0.396,0.709,0.104
3,0,0.564,0.571,0.518,0.543,0.518,0.611,0.129
4,1,0.572,0.590,0.472,0.524,0.472,0.672,0.144
5,2,0.589,0.600,0.534,0.565,0.534,0.645,0.179
6,0,0.604,0.601,0.616,0.609,0.616,0.591,0.208
7,1,0.600,0.606,0.571,0.588,0.571,0.629,0.199
8,2,0.649,0.720,0.486,0.580,0.486,0.811,0.297
9,0,0.622,0.610,0.678,0.642,0.678,0.566,0.245


In [11]:
metrics_df_CNN.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/TF Domain/CNN/Delta_tf_CNN.csv', index = False)

# GRU

In [12]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model_gru(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(GRU(256, return_sequences=True))
    model.add(GRU(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning_gru(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model_gru(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model_gru(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Delta/CNN_GRU/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Delta/CNN_GRU/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)



In [13]:
global_model_gru, metrics_df_gru = federated_learning_gru(Theta_data)

Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 37, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 256)           0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 14s 108ms/step - loss: 1.7025 - accuracy: 0.5027 - val_loss: 1.6974 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 1s 30ms/step - loss: 1.6922 - accuracy: 0.5032 - val_loss: 1.6873 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 1s 23ms/step - loss: 1.6820 - accuracy: 0.5038 - val_loss: 1.6773 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 1s 21ms/step - loss: 1.6720 - accuracy: 0.5038 - val_loss: 1.6674 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 1s 24ms/step - loss: 1.6620 - accuracy: 0.5038 - val_loss: 1.6576 - val_accuracy: 0.4849
Epoch 6/100
29/29 [==============================] - 1s 24ms/step - loss: 1.6521 - accuracy: 0.5038 - val_loss: 1.6479 - val_accuracy: 0.4849
Epoch 7/100
29/29 [==============================] - 1s 29ms/step - loss: 1.6424 - accuracy: 0.5038 - val_loss: 1.6382 - val_accuracy: 0.4849
Epoch 8/100
29/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
28/28 [==============================] - 0s 17ms/step - loss: 1.6926 - accuracy: 0.5011 - val_loss: 1.6879 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 1s 24ms/step - loss: 1.6827 - accuracy: 0.5017 - val_loss: 1.6782 - val_accuracy: 0.5305
Epoch 4/100
28/28 [==============================] - 1s 24ms/step - loss: 1.6729 - accuracy: 0.5017 - val_loss: 1.6687 - val_accuracy: 0.5373
Epoch 5/100
28/28 [==============================] - 0s 17ms/step - loss: 1.6632 - accuracy: 0.5011 - val_loss: 1.6592 - val_accuracy: 0.5181
Epoch 6/100
28/28 [==============================] - 0s 18ms/step - loss: 1.6535 - accuracy: 0.5037 - val_loss: 1.6498 - val_accuracy: 0.5249
Epoch 7/100
28/28 [==============================] - 0s 16ms/step - loss: 1.6440 - accuracy: 0.5110 - val_loss: 1.6405 - val_accuracy: 0.5204
Epoch 8/100
28/28 [==============================] - 0s 16ms/step - loss: 1.6346 - accuracy: 0.5119 - val_loss: 1.6313 - val_accuracy: 0.5204
Epoch 

...

29/29 [==============================] - 8s 63ms/step - loss: 0.5325 - accuracy: 0.7858 - val_loss: 0.8284 - val_accuracy: 0.4817
Epoch 2/100
29/29 [==============================] - 1s 24ms/step - loss: 0.5423 - accuracy: 0.7826 - val_loss: 0.9039 - val_accuracy: 0.4892
Epoch 3/100
29/29 [==============================] - 0s 14ms/step - loss: 0.5055 - accuracy: 0.8109 - val_loss: 0.8712 - val_accuracy: 0.4881
Epoch 4/100
29/29 [==============================] - 1s 24ms/step - loss: 0.5059 - accuracy: 0.8071 - val_loss: 0.8207 - val_accuracy: 0.4935
Epoch 5/100
29/29 [==============================] - 0s 15ms/step - loss: 0.4992 - accuracy: 0.8190 - val_loss: 0.8850 - val_accuracy: 0.4925
Epoch 6/100
29/29 [==============================] - 1s 23ms/step - loss: 0.5052 - accuracy: 0.8136 - val_loss: 0.8115 - val_accuracy: 0.5140
Epoch 7/100
29/29 [==============================] - 1s 23ms/step - loss: 0.5004 - accuracy: 0.8074 - val_loss: 0.8030 - val_accuracy: 0.5388
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 7s 58ms/step - loss: 0.5628 - accuracy: 0.7818 - val_loss: 0.8956 - val_accuracy: 0.5057
Epoch 2/100
28/28 [==============================] - 1s 33ms/step - loss: 0.5173 - accuracy: 0.8014 - val_loss: 0.8636 - val_accuracy: 0.5124
Epoch 3/100
28/28 [==============================] - 1s 32ms/step - loss: 0.5176 - accuracy: 0.8093 - val_loss: 0.7946 - val_accuracy: 0.5362
Epoch 4/100
28/28 [==============================] - 1s 33ms/step - loss: 0.5098 - accuracy: 0.8033 - val_loss: 0.7757 - val_accuracy: 0.6131
Epoch 5/100
28/28 [==============================] - 1s 21ms/step - loss: 0.5052 - accuracy: 0.8070 - val_loss: 0.7735 - val_accuracy: 0.5588
Epoch 6/100
28/28 [==============================] - 1s 33ms/step - loss: 0.5036 - accuracy: 0.8147 - val_loss: 0.7661 - val_accuracy: 0.6290
Epoch 7/100
28/28 [==============================] - 1s 22ms/step - loss: 0.5101 - accuracy: 0.8039 - val_loss: 0.7617 - val_accuracy: 0.5905
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 9s 53ms/step - loss: 0.5726 - accuracy: 0.7698 - val_loss: 0.8102 - val_accuracy: 0.5021
Epoch 2/100
31/31 [==============================] - 1s 24ms/step - loss: 0.5333 - accuracy: 0.7941 - val_loss: 0.8207 - val_accuracy: 0.5031
Epoch 3/100
31/31 [==============================] - 1s 17ms/step - loss: 0.5488 - accuracy: 0.7744 - val_loss: 0.8601 - val_accuracy: 0.4990
Epoch 4/100
31/31 [==============================] - 1s 24ms/step - loss: 0.5466 - accuracy: 0.7907 - val_loss: 0.8184 - val_accuracy: 0.5134
Epoch 5/100
31/31 [==============================] - 1s 24ms/step - loss: 0.5406 - accuracy: 0.7845 - val_loss: 0.7792 - val_accuracy: 0.6105
Epoch 6/100
31/31 [==============================] - 0s 15ms/step - loss: 0.5200 - accuracy: 0.7946 - val_loss: 0.7760 - val_accuracy: 0.6085
Epoch 7/100
31/31 [==============================] - 1s 24ms/step - loss: 0.5245 - accuracy: 0.7866 - val_loss: 0.7690 - val_accuracy: 0.6126
Epoch 8/100
31/31 

In [14]:
metrics_df_gru

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.556114,0.603715,0.326633,0.423913,0.326633,0.785595,0.112228
1,1,0.550847,0.577922,0.377119,0.456410,0.377119,0.724576,0.101695
2,2,0.550201,0.565104,0.435743,0.492063,0.435743,0.664659,0.100402
3,0,0.565327,0.558735,0.621441,0.588422,0.621441,0.509213,0.130653
4,1,0.569209,0.606987,0.392655,0.476844,0.392655,0.745763,0.138418
5,2,0.567269,0.652968,0.287149,0.398884,0.287149,0.847390,0.134538
6,0,0.601340,0.596184,0.628141,0.611746,0.628141,0.574539,0.202680
7,1,0.593927,0.595409,0.586158,0.590747,0.586158,0.601695,0.187853
8,2,0.632530,0.653488,0.564257,0.605603,0.564257,0.700803,0.265060
9,0,0.633166,0.661914,0.544389,0.597426,0.544389,0.721943,0.266332


In [15]:
metrics_df_gru.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/TF Domain/GRU/Delta_tf_GRU.csv', index = False)

In [20]:
print('hello')

hello
